In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2024-07-22 02:53:55.458453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
img_height, img_width = 48, 48
batch_size = 32


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'train',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='grayscale',  
    label_mode='categorical'  
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'test',
    image_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='grayscale',
    label_mode='categorical'
)

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [3]:
def create_custom_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax') 
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [4]:
model = create_custom_model()

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

model.save('custom_model.h5')


Epoch 1/10


/Users/mohammed/miniconda3/envs/MLenv2/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


898/898 ━━━━━━━━━━━━━━━━━━━━ 104s 113ms/step - accuracy: 0.2378 - loss: 2.3723 - val_accuracy: 0.3749 - val_loss: 1.6272
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 98s 109ms/step - accuracy: 0.3524 - loss: 1.6330 - val_accuracy: 0.4270 - val_loss: 1.4907
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 102s 113ms/step - accuracy: 0.4181 - loss: 1.5087 - val_accuracy: 0.4570 - val_loss: 1.4171
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 104s 116ms/step - accuracy: 0.4437 - loss: 1.4343 - val_accuracy: 0.4675 - val_loss: 1.3791
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 140s 114ms/step - accuracy: 0.4679 - loss: 1.3827 - val_accuracy: 0.4745 - val_loss: 1.3331
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 138s 109ms/step - accuracy: 0.4891 - loss: 1.3440 - val_accuracy: 0.4900 - val_loss: 1.3132
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 137s 153ms/step - accuracy: 0.4946 - loss: 1.3119 - val_accuracy: 0.5052 - val_loss: 1.2878
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 115s 128ms/step - accuracy: 0.5079 - loss: 1.275